In [2]:
import pandas as pd
df = pd.read_csv('./clean_vote.csv')
df.head()

,Unnamed: 0,text,vote,help
0,0,"['Refined gameplay, with layers of mechanics, ...",Recommended,['14 people found this review helpful']
1,1,"[""There are complaints in the reviews about th...",Recommended,['9 people found this review helpful']
2,2,"[""Clicker game, time management idle game, or ...",Recommended,['7 people found this review helpful']
3,3,"[""I've been playing this game for over a year ...",Recommended,['26 people found this review helpful4 people ...
4,4,"[""I've been playing idle/clicker/incremental g...",Recommended,['10 people found this review helpful']


In [21]:
recommend=df[df['vote'] == "Recommended"]
recommend=recommend.reset_index() 
recommend.head()

,index,Unnamed: 0,text,vote,help
0,0,0,"['Refined gameplay, with layers of mechanics, ...",Recommended,['14 people found this review helpful']
1,1,1,"[""There are complaints in the reviews about th...",Recommended,['9 people found this review helpful']
2,2,2,"[""Clicker game, time management idle game, or ...",Recommended,['7 people found this review helpful']
3,3,3,"[""I've been playing this game for over a year ...",Recommended,['26 people found this review helpful4 people ...
4,4,4,"[""I've been playing idle/clicker/incremental g...",Recommended,['10 people found this review helpful']


In [22]:
recommend.shape

(80159, 5)

In [23]:
recommend=recommend[:10000]

In [24]:
recommend.shape

(10000, 5)

In [25]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

        # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs
recommend['preprocessing'] = docs_preprocessor(recommend['text'])

Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [26]:
from gensim.models import Phrases
# Add bigrams to docs (only ones that appear 10 times or more).
docs=recommend['preprocessing']
bigram = Phrases(docs, min_count=10)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [27]:
#Remove rare and common tokens:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 85% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.85)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 30032
Number of unique words after removing rare and common words: 5621


In [28]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [29]:
from gensim.models import LdaModel
# Set training parameters.
num_topics = 10
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)


CPU times: user 3min 35s, sys: 10.9 s, total: 3min 46s
Wall time: 2min 29s


In [30]:
import pprint
pprint.pprint(model.print_topics())

[(0,
  '0.057*"unit" + 0.044*"space" + 0.043*"faction" + 0.042*"building" + '
  '0.028*"battle" + 0.028*"base" + 0.028*"campaign" + 0.024*"system" + '
  '0.021*"race" + 0.015*"early"'),
 (1,
  '0.026*"with" + 0.020*"your" + 0.014*"which" + 0.013*"weapon" + 0.012*"from" '
  '+ 0.012*"enemy" + 0.011*"different" + 0.011*"level" + 0.010*"them" + '
  '0.009*"their"'),
 (2,
  '0.157*"life" + 0.134*"half_life" + 0.034*"tomb" + 0.027*"valve" + '
  '0.023*"raider" + 0.019*"humor" + 0.019*"fortress" + 0.017*"team_fortress" + '
  '0.016*"season" + 0.013*"sequel"'),
 (3,
  '0.058*"spell" + 0.034*"dungeon" + 0.027*"item" + 0.027*"titan" + '
  '0.024*"monster" + 0.021*"wizard" + 0.019*"titan_quest" + 0.019*"coop" + '
  '0.016*"relic" + 0.013*"rogue"'),
 (4,
  '0.059*"shadow" + 0.035*"shadow_warrior" + 0.032*"wang" + 0.031*"borderland" '
  '+ 0.028*"gun" + 0.022*"were" + 0.016*"mouse" + 0.014*"chernobyl" + '
  '0.013*"original" + 0.013*"ammo"'),
 (5,
  '0.031*"that" + 0.019*"have" + 0.018*"with" + 0.

In [31]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pyLDAvis.gensim.prepare(model, corpus, dictionary)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.207560 -0.013046       1        1  53.927303
1     -0.173798  0.150441       2        1  16.554482
7     -0.225858 -0.249194       3        1  12.420083
6     -0.178105  0.075212       4        1   7.410005
0      0.029490  0.331311       5        1   3.159002
8      0.026781 -0.158101       6        1   2.199934
4      0.172504 -0.019434       7        1   1.516772
3      0.184408 -0.009472       8        1   1.190487
9      0.177096 -0.062625       9        1   0.835621
2      0.195042 -0.045092      10        1   0.786315, topic_info=     Category          Freq               Term         Total  loglift  logprob
term                                                                          
30    Default  26406.000000               game  26406.000000  30.0000  30.0000
79    Default  16384.000000               this  16384.000000  29.0000  29.0000
50    Default   3697.000000               play   3697.000000  28.0000  28.0000
856   Default   2630.000000              story   2630.000000  27.0000  27.0000
541   Default   1392.000000               life   1392.000000  26.0000  26.0000
147   Default   4236.000000               good   4236.000000  25.0000  25.0000
199   Default   3706.000000               very   3706.000000  24.0000  24.0000
148   Default   3123.000000              great   3123.000000  23.0000  23.0000
2403  Default   1286.000000               unit   1286.000000  22.0000  22.0000
73    Default  12937.000000               that  12937.000000  21.0000  21.0000
93    Default   6306.000000               your   6306.000000  20.0000  20.0000
420   Default   2670.000000               well   2670.000000  19.0000  19.0000
373   Default   1882.000000             played   1882.000000  18.0000  18.0000
71    Default   1741.000000             system   1741.000000  17.0000  17.0000
263   Default   2046.000000          character   2046.000000  16.0000  16.0000
670   Default   1120.000000               were   1120.000000  15.0000  15.0000
90    Default  10871.000000               with  10871.000000  14.0000  14.0000
110   Default   4469.000000               just   4469.000000  13.0000  13.0000
5461  Default    754.000000          half_life    754.000000  12.0000  12.0000
243   Default   1757.000000               best   1757.000000  11.0000  11.0000
115   Default   1747.000000            playing   1747.000000  10.0000  10.0000
32    Default   1876.000000               hour   1876.000000   9.0000   9.0000
1734  Default   1003.000000           building   1003.000000   8.0000   8.0000
1486  Default    994.000000              space    994.000000   7.0000   7.0000
31    Default   1555.000000           gameplay   1555.000000   6.0000   6.0000
1761  Default    965.000000            faction    965.000000   5.0000   5.0000
69    Default    925.000000              steam    925.000000   4.0000   4.0000
802   Default   1550.000000             combat   1550.000000   3.0000   3.0000
391   Default   1530.000000               love   1530.000000   2.0000   2.0000
231   Default    865.000000               item    865.000000   1.0000   1.0000
...       ...           ...                ...           ...      ...      ...
5449  Topic10    151.819458              valve    152.819702   4.8390  -3.6109
4028  Topic10    128.238037             raider    129.238251   4.8378  -3.7797
3391  Topic10    109.385933              humor    110.386787   4.8365  -3.9387
3060  Topic10    106.666840           fortress    107.667175   4.8362  -3.9638
5517  Topic10     93.644600      team_fortress     94.644943   4.8349  -4.0940
4998  Topic10     92.648232             season     93.648788   4.8348  -4.1047
5223  Topic10     73.203224               lara     74.203415   4.8320  -4.3403
4578  Topic10     72.041855            prequel     73.044098   4.8318  -4.3563
1592  Topic10     71.593979               rise     72.594208   4.8317 

In [5]:
notrecommend=df.loc[df['vote'] == "Not Recommended"]
notrecommend=notrecommend.reset_index() 
notrecommend=notrecommend[:10000]

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
%pylab inline

import pandas as pd
import pickle as pk
from scipy import sparse as sp

def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

        # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isdigit()] for doc in docs]
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 3] for doc in docs]
    
    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
  
    return docs
notrecommend['preprocessing'] = docs_preprocessor(notrecommend['text'])

Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [7]:
from gensim.models import Phrases
# Add bigrams to docs (only ones that appear 10 times or more).
docs=notrecommend['preprocessing']
bigram = Phrases(docs, min_count=10)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [16]:
#Remove rare and common tokens:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 85% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.85)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 30635
Number of unique words after removing rare and common words: 5751


In [17]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [18]:
from gensim.models import LdaModel
# Set training parameters.
num_topics = 10
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model2 = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)


CPU times: user 3min 46s, sys: 10.6 s, total: 3min 56s
Wall time: 2min 42s


In [19]:
import pprint
pprint.pprint(model2.print_topics())

[(0,
  '0.037*"your" + 0.017*"level" + 0.015*"enemy" + 0.013*"weapon" + '
  '0.012*"unit" + 0.012*"item" + 0.012*"each" + 0.011*"them" + 0.011*"system" '
  '+ 0.010*"skill"'),
 (1,
  '0.037*"quest" + 0.022*"building" + 0.018*"kingdom" + 0.016*"island" + '
  '0.011*"build" + 0.011*"management" + 0.011*"resource" + 0.010*"need" + '
  '0.010*"land" + 0.009*"there"'),
 (2,
  '0.035*"game" + 0.026*"bug" + 0.026*"this" + 0.025*"issue" + 0.024*"update" '
  '+ 0.023*"still" + 0.017*"version" + 0.015*"crash" + 0.015*"patch" + '
  '0.012*"play"'),
 (3,
  '0.173*"player" + 0.043*"dont" + 0.037*"friend" + 0.033*"server" + '
  '0.031*"community" + 0.027*"multiplayer" + 0.024*"cant" + 0.024*"team" + '
  '0.020*"play" + 0.018*"against"'),
 (4,
  '0.050*"trash" + 0.049*"video" + 0.043*"mafia" + 0.041*"watch" + '
  '0.039*"http" + 0.033*"controller" + 0.026*"waste_time" + 0.025*"youtube" + '
  '0.025*"mobile" + 0.021*"micro"'),
 (5,
  '0.036*"money" + 0.036*"year" + 0.026*"content" + 0.024*"they" + '
 

In [20]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pyLDAvis.gensim.prepare(model2, corpus, dictionary)

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.231580 -0.084506       1        1  60.041363
9      0.219302  0.073328       2        1  12.425448
0      0.212413  0.119444       3        1   8.548258
2      0.094488 -0.266343       4        1   6.735573
5     -0.042668 -0.251316       5        1   3.890231
1      0.031037  0.259970       6        1   3.642544
3     -0.235050 -0.031749       7        1   1.870894
7     -0.220724 -0.044046       8        1   1.651660
4     -0.148527  0.111629       9        1   0.673910
6     -0.141851  0.113589      10        1   0.520120, topic_info=     Category          Freq          Term         Total  loglift  logprob
term                                                                     
37    Default  32171.000000          game  32171.000000  30.0000  30.0000
81    Default  19719.000000          this  19719.000000  29.0000  29.0000
424   Default   2894.000000        player   2894.000000  28.0000  28.0000
261   Default   7721.000000          they   7721.000000  27.0000  27.0000
87    Default   6651.000000          your   6651.000000  26.0000  26.0000
164   Default   4710.000000          play   4710.000000  25.0000  25.0000
195   Default   2516.000000          than   2516.000000  24.0000  24.0000
1332  Default   1648.000000          card   1648.000000  23.0000  23.0000
107   Default   6650.000000          like   6650.000000  22.0000  22.0000
446   Default   1710.000000        better   1710.000000  21.0000  21.0000
109   Default   2364.000000         money   2364.000000  20.0000  20.0000
652   Default   2448.000000          feel   2448.000000  19.0000  19.0000
674   Default   2396.000000         still   2396.000000  18.0000  18.0000
100   Default  16096.000000          that  16096.000000  17.0000  17.0000
116   Default   3161.000000          very   3161.000000  16.0000  16.0000
80    Default   6112.000000         there   6112.000000  15.0000  15.0000
239   Default   2045.000000     character   2045.000000  14.0000  14.0000
361   Default   2099.000000          been   2099.000000  13.0000  13.0000
585   Default   2070.000000         their   2070.000000  12.0000  12.0000
150   Default   1500.000000         issue   1500.000000  11.0000  11.0000
580   Default   1858.000000         story   1858.000000  10.0000  10.0000
1567  Default   1426.000000           bug   1426.000000   9.0000   9.0000
598   Default   1124.000000          year   1124.000000   8.0000   8.0000
2030  Default   1093.000000         quest   1093.000000   7.0000   7.0000
212   Default   3704.000000        really   3704.000000   6.0000   6.0000
1494  Default   1288.000000        update   1288.000000   5.0000   5.0000
188   Default   1957.000000          need   1957.000000   4.0000   4.0000
38    Default   1703.000000      gameplay   1703.000000   3.0000   3.0000
78    Default   2971.000000          them   2971.000000   2.0000   2.0000
104   Default   4426.000000          will   4426.000000   1.0000   1.0000
...       ...           ...           ...           ...      ...      ...
4073  Topic10    210.723862        hacker    211.640900   5.2545  -2.9827
2196  Topic10    130.795395        family    131.712814   5.2519  -3.4596
503   Topic10    127.537666         elite    128.454773   5.2517  -3.4848
2195  Topic10    121.856567    everywhere    122.773689   5.2514  -3.5304
1442  Topic10     92.477646      keyboard     93.394691   5.2490  -3.8062
1063  Topic10     75.272469    doesn_work     76.189697   5.2468  -4.0121
5519  Topic10     74.406723  space_marine     75.324127   5.2466  -4.0237
5718  Topic10     68.690849          dow2     69.608261   5.2456  -4.1036
4126  Topic10     67.933174          body     68.850319   5.2455  -4.1147
4828  Topic10     65.782104         moral     66.699654   5.2450  -4.1469
4807  Topic10     62.634705         horse     63.551849   5.2443  -4.1959
3548  Topic10     60.351337       related     61.2685